In [ ]:
import pickle
import os
import re
import nltk
import glob
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

stops = set(stopwords.words("english"))

In [ ]:
def SplitTextIntoSentences(txt):
    dtxt=re.sub(r"\.\s|\.\n",".天官赐福,百无禁忌%@!^&ЁЪЫї",txt)
    etxt=re.sub(r"\!\s|\!\n","!天官赐福,百无禁忌%@!^&ЁЪЫї",dtxt)
    qtxt=re.sub(r"\?\s|\?\n","?天官赐福,百无禁忌%@!^&ЁЪЫї",etxt)
    sentences= re.split(r'\天\官\赐\福\,\百\无\禁\忌\%\@\!\^\&\Ё\Ъ\Ы\ї', qtxt)

    return sentences

In [ ]:
email = '[a-zA-Z0-9]+@\w+[\.\w+]+'
money='\$[0-9]+[\.[0-9]+]?[k]?'
date='[0-9]{1,2}\s[a-zA-Z]+\s[0-9]{1,4}|[0-9]{1,2}\s[0-9]{1,2}\s[0-9]{1,4}|[0-9]{1,2}\S[0-9]{1,2}\S[0-9]{1,4}|[a-zA-Z]+\s[0-9]{1,2}\S?\s[0-9]{1,4}'
phone='\([0-9]{3}\)\s[0-9]{3}\-[0-9]{4}'
words='[a-zA-z]+[\'[a-z]+]?|\S'
prop_names=''
full_exp=email+'|'+money+'|'+date+'|'+phone+'|'+words



tokenize_regex = re.compile(full_exp, re.I)

In [ ]:
dictionary=dict()

In [ ]:
with open('data\\dictionary.pkl', "rb") as fp:
        dictionary = pickle.load(fp)

In [ ]:
def w2vec_vectorize(text: str, model):
    sentence_list = SplitTextIntoSentences(text)
    sent_vectors = []
    for sentence in sentence_list:
        tokens=tokenize_regex.findall(sentence)
        word_vectors = []
        for token in tokens:
            if token not in stops:
                try:
                    word_vectors.append(model.wv[token.lower()])
                except Exception as e:
                    print(e)

        sent_vector = np.zeros(model.vector_size)
        if (len(word_vectors) > 0):
            sent_vector = (np.array([sum(x) for x in zip(*word_vectors)])) / sent_vector.size
        sent_vectors.append(sent_vector)

    vector = np.zeros(model.vector_size)
    if (len(sent_vectors) > 0):
        vector = (np.array([sum(x) for x in zip(*sent_vectors)])) / vector.size

    return vector

In [ ]:
def sentence_list(rootDir):
    sentence_list = [] 
    for lists in os.listdir(rootDir):
      path = os.path.join(rootDir, lists)
      for filename in glob.glob(path+'/*'):
        with open(os.path.join(os.getcwd(), filename), 'r',errors='ignore') as file:
          sentences=SplitTextIntoSentences(file.read().rstrip())
          for s in sentences:
            tokens=tokenize_regex.findall(s)
            tokens_filtred = [t for t in tokens if not re.match(r'\W|\d',t) and t not in stops]
            sentence_list.append(tokens_filtred)
    #print(sentence_list)
    with open("data\\sentence_list", "wb") as file:
        pickle.dump(sentence_list, file)

In [ ]:
sentence_list("D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\data\\train")

In [ ]:
with open("data\\sentence_list", "rb") as file:
     sentences = pickle.load(file)

In [ ]:
sentences

In [ ]:
import gensim.models

w2v =  gensim.models.Word2Vec(sentences=sentences, window=10, vector_size=100, epochs=5)

w2v.save('data\\w2v_model')